In [2]:
import rasterio
import numpy as np

def find_valid_coordinates(raster_path, sample_size=5):
    """
    Finds coordinates where the raster has non-zero values.
    
    Parameters:
    raster_path (str): Path to the .asc raster file.
    sample_size (int): Number of valid coordinates to return.
    
    Returns:
    list: List of (lat, lon) tuples.
    """
    with rasterio.open(raster_path) as dataset:
        data = dataset.read(1)  # Read raster band
        transform = dataset.transform

        # Find all non-zero pixels
        rows, cols = np.where(data > 0)

        if len(rows) == 0:
            print("No non-zero values found in the raster.")
            return []

        # Sample some valid locations
        sample_indices = np.random.choice(len(rows), min(sample_size, len(rows)), replace=False)
        
        valid_coords = []
        for idx in sample_indices:
            row, col = rows[idx], cols[idx]
            x, y = transform * (col, row)  # Convert raster index to projected coordinates
            valid_coords.append((x, y))

        return valid_coords

# Paths to your rasters
kharif_raster = "/Users/preitish/agrihackathon/kharif.asc"
rabi_raster = "/Users/preitish/agrihackathon/rabi.asc"

# Find valid points
kharif_coords = find_valid_coordinates(kharif_raster)
rabi_coords = find_valid_coordinates(rabi_raster)

print("Kharif Crop Coordinates:", kharif_coords)
print("Rabi Crop Coordinates:", rabi_coords)

Kharif Crop Coordinates: [(np.float64(1580000.0), np.float64(1845000.0)), (np.float64(1555000.0), np.float64(2300000.0)), (np.float64(2310000.0), np.float64(1540000.0)), (np.float64(1940000.0), np.float64(1430000.0)), (np.float64(3480000.0), np.float64(2720000.0))]
Rabi Crop Coordinates: [(np.float64(2390000.0), np.float64(1975000.0)), (np.float64(3460000.0), np.float64(2440000.0)), (np.float64(1640000.0), np.float64(2775000.0)), (np.float64(2540000.0), np.float64(2530000.0)), (np.float64(1735000.0), np.float64(2435000.0))]


In [4]:
from pyproj import Transformer

def convert_to_latlon(x, y, src_crs="EPSG:32643"):
    transformer = Transformer.from_crs(src_crs, "EPSG:4326", always_xy=True)
    lon, lat = transformer.transform(x, y)
    return lat, lon

# Kharif Crop Coordinates (UTM)
kharif_coords = [
    (2055000.0, 1655000.0),
    (2625000.0, 2145000.0),
    (1950000.0, 1890000.0),
    (2430000.0, 2430000.0),
    (3600000.0, 1285000.0),
]

# Rabi Crop Coordinates (UTM)
rabi_coords = [
    (2130000.0, 1205000.0),
    (2120000.0, 2640000.0),
    (2265000.0, 1800000.0),
    (3600000.0, 2840000.0),
    (2380000.0, 1660000.0),
]

# Convert and print results
kharif_latlon = [convert_to_latlon(x, y) for x, y in kharif_coords]
rabi_latlon = [convert_to_latlon(x, y) for x, y in rabi_coords]

print("Kharif Lat/Lon:", kharif_latlon)
print("Rabi Lat/Lon:", rabi_latlon)

Kharif Lat/Lon: [(14.523144351233627, 89.30036236779829), (18.325933412535278, 94.78683967459526), (16.646373768183857, 88.49111636860658), (20.952743423742714, 93.32743838165601), (10.351120199864093, 102.29278033785417)]
Rabi Lat/Lon: [(10.547667490250321, 89.74339466220817), (23.073185208914474, 90.67883198543565), (15.656903993491277, 91.27766966527379), (22.735716072734213, 104.2706105985252), (14.367554350105394, 92.2046650757869)]


In [5]:
import numpy as np
import rasterio

def compare_raster_differences(file1, file2):
    with rasterio.open(file1) as raster1, rasterio.open(file2) as raster2:
        data1 = raster1.read(1)
        data2 = raster2.read(1)

        # Compute absolute difference
        diff = np.abs(data1 - data2)

        # Find statistics
        max_diff = np.max(diff)
        mean_diff = np.mean(diff)
        nonzero_diffs = np.count_nonzero(diff)

    return {
        "Max Difference": max_diff,
        "Mean Difference": mean_diff,
        "Different Pixels Count": nonzero_diffs
    }

file1 = "/Users/preitish/agrihackathon/kharif.asc"
file2 = "/Users/preitish/agrihackathon/rabi.asc"

diff_results = compare_raster_differences(file1, file2)

# Print results
for key, value in diff_results.items():
    print(f"{key}: {value}")

Max Difference: 9871
Mean Difference: 431.85277694524495
Different Pixels Count: 111355


In [7]:
import rasterio
import numpy as np

def compare_rasters(file1, file2):
    """
    Compare two raster files to check if they are identical.

    Parameters:
    file1 (str): Path to the first raster file (.tif or .asc)
    file2 (str): Path to the second raster file (.tif or .asc)

    Returns:
    dict: A dictionary showing whether metadata and pixel values match.
    """
    results = {}

    with rasterio.open(file1) as raster1, rasterio.open(file2) as raster2:
        # Compare metadata (CRS, transform, dimensions)
        results["Same CRS"] = raster1.crs == raster2.crs
        results["Same Transform"] = raster1.transform == raster2.transform
        results["Same Dimensions"] = raster1.shape == raster2.shape
        results["Same NODATA Value"] = raster1.nodata == raster2.nodata

        # Read data as numpy arrays
        data1 = raster1.read(1)
        data2 = raster2.read(1)

        # Compare pixel values
        results["Same Pixel Values"] = np.array_equal(data1, data2)

    # Final decision
    results["Identical Rasters"] = all(results.values())

    return results

# Example usage
file1 = "/Users/preitish/agrihackathon/kharif.asc"
file2 = "/Users/preitish/agrihackathon/rabi.asc"

comparison_results = compare_rasters(file1, file2)

# Print results
for key, value in comparison_results.items():
    print(f"{key}: {value}")

Same CRS: True
Same Transform: True
Same Dimensions: True
Same NODATA Value: True
Same Pixel Values: False
Identical Rasters: False


In [12]:
import rasterio
from rasterio.transform import rowcol
from pyproj import Transformer, CRS
import glob
import os

def get_raster_values(lat, lon, raster_paths, default_crs="EPSG:32643"):
    """
    Fetches raster values at the given latitude and longitude for multiple .asc and .tif raster files.
    Also identifies files returning NoData values.

    Parameters:
    lat (float): Latitude coordinate
    lon (float): Longitude coordinate
    raster_paths (list): List of file paths to raster files (.asc, .tif)
    default_crs (str): Default CRS if the raster file lacks CRS info (default: "EPSG:32643")

    Returns:
    dict: Dictionary with filenames as keys and raster values as values
    list: List of files that returned NoData values
    """
    results = {}
    nodata_files = []  # List to store filenames with NoData values

    for raster_path in raster_paths:
        try:
            with rasterio.open(raster_path) as dataset:
                dataset_crs = dataset.crs if dataset.crs else CRS.from_string(default_crs)

                transformer = Transformer.from_crs("EPSG:4326", dataset_crs, always_xy=True)
                x, y = transformer.transform(lon, lat)

                row, col = rowcol(dataset.transform, x, y)

                # Read raster value
                value = dataset.read(1)[row, col]
                nodata_value = dataset.nodata  # Get NoData value from metadata

                # Extract filename without extension
                file_name = os.path.splitext(os.path.basename(raster_path))[0]

                # Check if the value matches NoData
                if value == nodata_value:
                    nodata_files.append(file_name)
                    results[file_name] = "NoData"
                else:
                    results[file_name] = value

        except Exception as e:
            results[os.path.basename(raster_path)] = f"Error: {str(e)}"

    return results, nodata_files

# Automatically get all .asc and .tif files from the folder
folder_path = "/Users/preitish/agrihackathon/"
raster_files = glob.glob(os.path.join(folder_path, "*.asc")) + glob.glob(os.path.join(folder_path, "*.tif"))

# Latitude & Longitude of interest
lat, lon = 28.6139, 77.2090  # Change to desired location

# Get raster values and identify NoData files
raster_values, nodata_files = get_raster_values(lat, lon, raster_files, "EPSG:32643")

# Print results
print("\n📌 **Raster Values:**")
for key, value in raster_values.items():
    print(f"{key}: {value}")

# Print NoData files separately
if nodata_files:
    print("\n⚠ **Files with NoData Values:**")
    for file in nodata_files:
        print(f"- {file}")
else:
    print("\n✅ No files returned NoData values!")


📌 **Raster Values:**
ffallow201314: 125
fclayskeletal: 0
floamy: 2824
fsoildep100_150: 2824
fwaterlog: 0
fsoildep25_50: 0
fsalt: 0
fnsa201314: 516
fsoildep150_200: 0
fwindero: 0
fsoildep75_100: 0
fsandy: 0
fwatero: 0
fsoildep50_75: 0
fkharif201314: 493
fclayey: 0
fsoildep0_25: 0
meantocd: 21.803150177001953
forest_fraction_5km_2013: 114321
frabi201314: 516
meanticd: 10.032190322875977
fire_SDAFD: NoData
fire_AFD: NoData
Forest_Fraction_5km_2013: 114321
fire_LPF: NoData

⚠ **Files with NoData Values:**
- fire_SDAFD
- fire_AFD
- fire_LPF
